In [1]:
pip install ydata-profiling[pyspark]

Note: you may need to restart the kernel to use updated packages.


In [1]:
from pyspark.sql import SparkSession
import numpy as np
import pandas as pd
from ydata_profiling import ProfileReport
from ydata_profiling import ProfileReport

In [2]:
# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/warehouse'

spark = SparkSession \
    .builder \
    .master("local[2]") \
    .appName("Python Spark profiling example") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .enableHiveSupport() \
    .getOrCreate()

In [3]:
hdfs_path = "hdfs://hdfs-nn:9000/demo/bronze/audience_reviews.csv"

In [4]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

schema = StructType([
    StructField("Show", StringType(), True),
    StructField("Rating", FloatType(), True),
    StructField("Review", StringType(), True),
])

audience_reviews_df = spark.read.csv(
    hdfs_path,
    header=True,
    schema=schema
)

In [5]:
audience_reviews_df.printSchema()
audience_reviews_df.show()

root
 |-- Show: string (nullable = true)
 |-- Rating: float (nullable = true)
 |-- Review: string (nullable = true)

+----+------+--------------------+
|Show|Rating|              Review|
+----+------+--------------------+
|Test|   0.0|         Test review|
|Loot|   2.5|I love the stars ...|
|Loot|   4.5|Great show to wat...|
|Loot|   0.5|Another Apple tan...|
|Loot|   4.5|This is such a fu...|
|Loot|   0.5|Terrible show - d...|
|Loot|   1.0|I love Maya Rudol...|
|Loot|   1.5|I tried to enjoy ...|
|Loot|   0.5|Love ya Maya but ...|
|Loot|   4.0|So far I am lovin...|
|Loot|   3.5|Scoot over Gwynet...|
|Loot|   1.0|Had high hopes fo...|
|Loot|   3.5|Funny and nice go...|
|Loot|   0.5|Don't waste your ...|
|Loot|   5.0|totally hooked on...|
|Loot|   1.0|Boring tedious.  ...|
|Loot|   5.0|Maya Rudolph amaz...|
|Loot|   1.5|A very heavy hand...|
|Loot|   5.0|Its literally eve...|
|Loot|   5.0|Maya is so funny....|
+----+------+--------------------+
only showing top 20 rows



In [6]:
audience_reviews_df.describe(['Show','Rating','Review']).toPandas()

,summary,Show,Rating,Review
0,count,65521,65521,65521
1,mean,1517.1726907630523,3.5146975778757956,None
2,stddev,740.5713617416238,1.738238191990853,None
3,min,11.22.63,0.0,! WOW ! good season i love it ! i hope you guy...
4,max,iCarly,5.0,🦑🎮 Not close to perfect but still amazing. Cre...


In [7]:
#Select the columns to profile. 
df_to_profile=audience_reviews_df.select("Show","Rating","Review")

In [8]:
pdf = df_to_profile.toPandas()
report = ProfileReport(pdf)


In [9]:
#save profiling report in a notebook
report.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 3/3 [01:43<00:00, 34.57s/it]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
#save profiling report in a file
report.to_file('audience_reviews-data_profiling.html')

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
#close spark session
spark.stop()